# Kruskal

Un altro algoritmo che ci permette di calcolare il minimo albero ricoprente di un grafo è quello di Kruskal. A differenza dell'algoritmo di Prim non rilassa la condizione di *contenere tutti i nodi*, piuttosto **rilassa la condizione di connettività**.

- <u>Spazio delle soluzioni</u>: grafi aciclici non orientati non connessi $\implies$ una foresta

    Se $G = < N, E, \lambda >$ è il grafo di parteza, lo spazio delle soluzioni è $F = < N, E', \lambda' >$, $E' \in E$

- <u>Soluzione iniziale</u>: $F_0 = < N, \emptyset, \lambda >$

- <u>Regola di movimento</u>: $move: F = < N, E_F, \lambda >\quad \to \quad F' = < N, E_F \cup e, \lambda$ >, con $e \in E - E_F$

- <u>Funzione di costo</u>: somma dei pesi sugli archi
- <u>Terminazione</u>: quando ho raggiunto $n-1$ archi

### Spiegazione dell'algoritmo e della sua complessità

1. Si prende la lista degli archi e si ordina in base al peso $O(m \cdot log(m)) \to O(m \cdot log(n))$
2. Si scorre la lista degli archi e bisogna verificare per ognuno di essi se chiude un ciclo
    Si considera l'arco $a = (x,y)$ e si verifica se esiste un arco nella soluzione corrente che parte da $a.y$
    - Se sì $\to$ allora l'arco $a$ chiude un ciclo
    - Se no, si aggiunge alla soluzione
    La complessità del passo 2 è pertanto $O(m \cdot n)$ considerando che si fa la verifica per ogni arco del grafo originario e il numero di archi nella soluzione è al più $n-1$ (ricordando che un albero è un grafo connesso aciclico e quindi m = n-1)

Si può ridurre la complessità del passo 2 utilizzando una struttura dati UnionFind. Ricordando infatti che un ciclo si crea quando gli archi connettono nodi che fanno parte della stessa componente massimale potrei fare la find di $a.x$ e $a.y$. Su una struttura dati quickFind una find si fa in tempo $O(1)$ e poi si fa l'unione dei due insiemi se sono diversi (che su una struttura dati quickFind con bilanciamento ha costo $O(log(n))$). Considerando il fatto che si possono fare al più $n-1$ union, la complessità complessiva del passo 2 utilizzando una struttura dati quickFind con bilanciamento è $O(n\cdot log(n))$.

Questo significa che il costo complessivo dell'algoritmo è $O(m \cdot log(n) + n \cdot log(n)) = O(m \cdot log(n))$ che è la stessa complessità dell'algoritmo di Prim. Notiamo che in questo caso però il costo potrebbe essere ridotto a $O(n\cdot log(n))$ se riuscissimo a ordinare gli archi in tempo lineare (tipo con un radix sort), pertanto questo potrebbe essere un algoritmo migliore di quello di Prim.

In [3]:
from grafi.grafinopesati import GrafoNOP, GrafoNOLAP
from grafi.unionfind import UnionFind

def _peso(elem):
    return elem[2]

In [ ]:
def kruskal( g : GrafoNOP ) -> GrafoNOP:
    archiordinati = sorted(g.archi(), key=_peso)
    forest: UnionFind = UnionFind(g.n)
    result = GrafoNOP(g.n) # è un minimo albero ricoprente
    count = 0 # archi aggiunti alla soluzione finale
    for x, y, p in archiordinati:
        if forest.find(x) != forest.find(y): # se non chiude un ciclo
            result.append((x, y, p))
            forest.union(forest.find(x), forest.find(y))
            count += 1
        if count == g.n - 1:
            return result
    return []

Segue il codice del professore in Java in cui ci sono dei commentini *carini* sulle complessità

In [ ]:
public Grafo<ArcoPesato> kruskal(){ // theta( m n) || (union find) theta(m lg n)
		ArrayList<ArcoPesato> archi = generaArchiOrdinati(); // theta(m lg n) || theta(n^2 + m lg n)
		int inseriti = 0;
		GrafoLista<ArcoPesato> albero = new GrafoLista<ArcoPesato>(n());
		// creare union find - theta(n)
		
		//theta (m + n lg n)
		for(int i=0; (i<archi.size())&& (inseriti<n()-1); i++){ // m volte
			List<Integer> lista = albero.depthFirstSearch(archi.get(i).getIn()); // theta(n) || theta (n^2)
			// 2 find theta(1)
			if (!lista.contains(archi.get(i).getFin())){ // theta(1)
				albero.aggiungiArco(archi.get(i)); // theta(1)
				inseriti++; // theta(1)
				// union degli insiemi che contengono archi[i].getFin()
				// e archi[i].getIn() - theta(n)
				// complessivamente facciamo n-1 union che hanno un costo theta(n lg n)
			}
		}
		if (inseriti==n()-1)
			return albero;
		return null;
	}

In [ ]:
public List<ArcoPesato> kruskalUF(){ //theta(m lg n)
		ArrayList<ArcoPesato> archi = generaArchiOrdinati(); // theta(m lg n) 
		int inseriti = 0;
		UnionFind foresta = new UnionFind(n());
		// creare union find - theta(n)
		List<ArcoPesato> risultato = new LinkedList<ArcoPesato>();
		//theta (m + n lg n)
		for(int i=0; (i<archi.size())&& (inseriti<n()-1); i++){ // m volte
			// 2 find theta(1)
			if (foresta.find(archi.get(i).getIn())!=foresta.find(archi.get(i).getFin())){ // theta(1)
				foresta.merge(foresta.find(archi.get(i).getIn()),foresta.find(archi.get(i).getFin())); // theta(1)
				inseriti++; // theta(1)
				risultato.add(archi.get(i));
				// union degli insiemi che contengono archi[i].getFin()
				// e archi[i].getIn() - theta(n)
				// complessivamente facciamo n-1 union che hanno un costo theta(n lg n)
			}
		}
		if (inseriti==n()-1)
			return risultato;
		return risultato;
	}